In [ ]:
# -----------------------
# 0. Import Libraries
# -----------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, make_scorer
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay


# -----------------------
# 1. Load & Merge Data
# -----------------------
novelty_df = pd.read_csv('novelty_scores.csv')
coherence_df = pd.read_csv('coherence_scores.csv')
distinctiveness_df = pd.read_csv('distinctiveness_scores.csv', index_col=0)

# Prepare distinctiveness summary
distinctiveness_summary = distinctiveness_df.reset_index()
distinctiveness_summary.rename(columns={'index' : 'category'}, inplace = True)
distinctiveness_summary = distinctiveness_summary[['category', 'final_distinctiveness_score']]


# Merge features
campaign_df = (
    novelty_df
    .merge(coherence_df[['category', 'coherence_score']], on='category', how='left')
    .merge(distinctiveness_summary, on='category', how='left')
)

# Rename for clarity
campaign_df.rename(columns={
    'coherence_score': 'coherence',
    'final_distinctiveness_score': 'distinctiveness',
    'normalized_mean_top_10_tfidf': 'novelty'
}, inplace=True)

# -----------------------
# 2. Date Processing & Campaign Duration
# -----------------------
for col in ['launch_date', 'success_date', 'failure_date']:
    campaign_df[col] = pd.to_datetime(campaign_df[col])

# Campaign duration in days
campaign_df['campaign_duration'] = campaign_df.apply(
    lambda row: (row['success_date'] - row['launch_date']).days
    if pd.notna(row['success_date'])
    else (row['failure_date'] - row['launch_date']).days,
    axis=1
)

# Seasonal encoding
campaign_df['day_number'] = (campaign_df['launch_date'] - campaign_df['launch_date'].min()).dt.days + 1
campaign_df['annual_cycle_sin'] = np.sin(2 * np.pi * campaign_df['day_number'] / 365)
campaign_df['annual_cycle_cos'] = np.cos(2 * np.pi * campaign_df['day_number'] / 365)

# -----------------------
# 3. Feature Engineering
# -----------------------
# Log-transform skewed numeric variables
campaign_df['goal_log'] = np.log10(campaign_df['funding_goal'])
campaign_df['description_length_log'] = np.log10(campaign_df['doc_length'])

# Standardize features
scaler = StandardScaler()
numeric_features = ['distinctiveness', 'coherence', 'novelty', 'goal_log', 'description_length_log', 'campaign_duration']
campaign_df[numeric_features] = scaler.fit_transform(campaign_df[numeric_features])

# Polynomial & interaction terms
campaign_df['coherence_sq'] = campaign_df['coherence']**2
campaign_df['distinctiveness_sq'] = campaign_df['distinctiveness']**2
campaign_df['novelty_sq'] = campaign_df['novelty']**2

campaign_df['distinctiveness_coherence'] = campaign_df['distinctiveness'] * campaign_df['coherence']
campaign_df['distinctiveness_novelty'] = campaign_df['distinctiveness'] * campaign_df['novelty']
campaign_df['novelty_coherence'] = campaign_df['novelty'] * campaign_df['coherence']

# -----------------------
# 4. Check Multicollinearity (VIF)
# -----------------------
X_vif = sm.add_constant(campaign_df[numeric_features + ['annual_cycle_sin', 'annual_cycle_cos']])
vif_df = pd.DataFrame({
    "Variable": X_vif.columns,
    "VIF": [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
})
print("VIF Scores:\n", vif_df)

# -----------------------
# 5. Prepare Modeling Data
# -----------------------
features = ['distinctiveness', 'coherence', 'novelty', 'annual_cycle_sin', 'annual_cycle_cos',
            'goal_log', 'description_length_log', 'campaign_duration', 'parent_category',
            'coherence_sq', 'distinctiveness_sq', 'novelty_sq',
            'distinctiveness_coherence', 'distinctiveness_novelty', 'novelty_coherence']

X = pd.get_dummies(campaign_df[features], columns=['parent_category'], drop_first=True, dtype=float)
y = campaign_df['success']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# -----------------------
# 6. Logistic Regression
# -----------------------
logreg = LogisticRegression(max_iter=500, class_weight='balanced')
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)

# Evaluation
print(f"Logistic Regression Accuracy: {accuracy_score(y_test, y_pred_lr):.3f}")
print(f"Logistic Regression F1: {f1_score(y_test, y_pred_lr):.3f}")

# Cross-validation
f1_scorer = make_scorer(f1_score)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_accuracy = cross_val_score(logreg, X, y, cv=cv, scoring='accuracy')
cv_f1 = cross_val_score(logreg, X, y, cv=cv, scoring=f1_scorer)
print("mean CV Accuracy:", np.mean(cv_accuracy))
print("mean CV F1:", np.mean(cv_f1))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_lr)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.show()

# -----------------------
# 7. Random Forest
# -----------------------
rf = RandomForestClassifier(n_estimators=300, max_depth=20, min_samples_leaf=10,
                            max_features='sqrt', class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.3f}")
print(f"Random Forest F1: {f1_score(y_test, y_pred_rf):.3f}")

# -----------------------
# 8. XGBoost
# -----------------------
xgb_model = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.01,
                              max_depth=9, min_child_weight=3,
                              colsample_bytree=0.5, subsample=0.5,
                              objective='binary:logistic', random_state=42, use_label_encoder=False,
                              eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
print(f"XGBoost Accuracy: {accuracy_score(y_test, y_pred_xgb):.3f}")
print(f"XGBoost F1: {f1_score(y_test, y_pred_xgb):.3f}")

# -----------------------
# 9. Sample PDP (Partial Dependence) for Key Features
# -----------------------
def compute_pdp(var, model, X_train, num_points=50):
    # Create a grid in standardized space with customized range for 'novelty'
    min_scaled = X_train[var].min()
    if var == 'novelty':
        max_scaled = min(X_train[var].max(), 12)
    else:
        max_scaled = X_train[var].max()
    grid_scaled = np.linspace(min_scaled, max_scaled, num_points)
    
    # Define the squared term and interaction terms for this variable
    squared_term = var + '_sq'
    interaction_terms = [
        col for col in X_train.columns
        if col != squared_term
        and var in col.split('_')
        and col != var
    ]
    
    # List of features to update
    features_to_set = [var, squared_term] + interaction_terms
    
    # Compute partial dependence
    pd_values = []
    for d_scaled in grid_scaled:
        X_temp = X_train.copy()
        X_temp[var] = d_scaled
        X_temp[squared_term] = d_scaled ** 2
        
        for inter_term in interaction_terms:
            other_var = [v for v in inter_term.split('_') if v != var][0]
            X_temp[inter_term] = d_scaled * X_train[other_var]
        
        # Predict probabilities and average them
        probs = model.predict_proba(X_temp)[:, 1]
        pd_values.append(np.mean(probs))
    
    return grid_scaled, pd_values


variables = ['distinctiveness', 'coherence', 'novelty']
new_labels = ['Distinctiveness', 'Coherence', 'Novelty']

# Set up the plot
font = {'size': 12}
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, var in enumerate(variables):
    grid_scaled, pd_values = compute_pdp(var, logreg, X_train)
    axes[i].plot(grid_scaled, pd_values, color='teal')
    axes[i].set_xlabel(new_labels[i], fontdict=font)
    axes[i].set_ylabel('Predicted Probability of Success', fontdict=font)

plt.tight_layout()
plt.subplots_adjust(wspace=0.2)
plt.show()

# -----------------------
# 9. Sample PDP (Partial Dependence) for interaction of Features
# -----------------------
def compute_2d_pdp(interaction, var1, var2, model, X_train, num_points=5):
    """Compute 2D partial dependence values for an interaction term."""
    
    # Create a meshgrid for the two variables with customized ranges
    if var1 == 'novelty':
        grid_var1 = np.linspace(X_train[var1].min(), min(X_train[var1].max(), 12), num_points)
    else:
        grid_var1 = np.linspace(X_train[var1].min(), X_train[var1].max(), num_points)
    
    if var2 == 'novelty':
        grid_var2 = np.linspace(X_train[var2].min(), min(X_train[var2].max(), 12), num_points)
    else:
        grid_var2 = np.linspace(X_train[var2].min(), X_train[var2].max(), num_points)
    
    grid_var1, grid_var2 = np.meshgrid(grid_var1, grid_var2)
    
    # Array to store partial dependence values
    pd_values = np.zeros_like(grid_var1)
    
    # Identify squared terms for var1 and var2
    squared_terms = [var + '_sq' for var in [var1, var2] if var + '_sq' in X_train.columns]
    
    # Identify other interactions (exclude squared-term interactions)
    other_interactions = [
        col for col in X_train.columns
        if '_' in col
        and col != interaction
        and (var1 in col.split('_') or var2 in col.split('_'))
        and not col.endswith('_sq')
    ]
    
    # Loop over the grid to compute partial dependence
    for i in range(num_points):
        for j in range(num_points):
            val1 = grid_var1[i, j]
            val2 = grid_var2[i, j]
            X_temp = X_train.copy()
            
            # Update the two variables being varied
            X_temp[var1] = val1
            X_temp[var2] = val2
            
            # Update squared terms if they exist
            if var1 + '_sq' in squared_terms:
                X_temp[var1 + '_sq'] = val1 ** 2
            if var2 + '_sq' in squared_terms:
                X_temp[var2 + '_sq'] = val2 ** 2
            
            # Update the main interaction term
            X_temp[interaction] = val1 * val2
            
            # Update other interactions involving var1 or var2
            for other_inter in other_interactions:
                vars_in_inter = [v for v in other_inter.split('_') if v != 'sq']
                if var1 in vars_in_inter:
                    other_var = [v for v in vars_in_inter if v != var1][0]
                    X_temp[other_inter] = val1 * X_train[other_var]
                elif var2 in vars_in_inter:
                    other_var = [v for v in vars_in_inter if v != var2][0]
                    X_temp[other_inter] = val2 * X_train[other_var]
            
            # Compute the average predicted probability
            probs = model.predict_proba(X_temp)[:, 1]
            pd_values[i, j] = np.mean(probs)
    
    return grid_var1, grid_var2, pd_values


# Define the interactions to plot using your actual column names
interactions = [
    ('distinctiveness_coherence', 'distinctiveness', 'coherence'),
    ('distinctiveness_novelty', 'distinctiveness', 'novelty'),
    ('novelty_coherence', 'novelty', 'coherence')
]

# Create a figure with three subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
font = {'size': 12}

# Generate and plot 2D PDPs for each interaction
for i, (interaction, var1, var2) in enumerate(interactions):
    grid_var1, grid_var2, pd_values = compute_2d_pdp(interaction, var1, var2, logreg, X_train)
    ax = axes[i]
    contour = ax.contourf(grid_var1, grid_var2, pd_values, levels=20, cmap='viridis')
    ax.set_xlabel(var1.capitalize(), fontdict=font)
    ax.set_ylabel(var2.capitalize(), fontdict=font)
    fig.colorbar(contour, ax=ax, label='Predicted Probability of Success')

plt.tight_layout()
plt.subplots_adjust(wspace=0.2)
plt.show()